<a target="_blank" href="https://colab.research.google.com/github/vanderbilt-data-science/ai_summer/blob/main/2_3-solns-functions-tools-agents.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tools, Agents, and Assistants
> For Vanderbilt University AI Summer 2024<br>Prepared by Dr. Charreau Bell

_Code versions applicable: May 17, 2024_

## Learning Outcomes
* Participants will understand schemas and algorithms for usage in function calling and tool calling applications and articulate the behavior.
* Participants will be able to describe the behavior of agents and articulate fundamental advantages and disadvantages this approach.
* Participants will be able to leverage schemas, tool calling, function calling, agents, API assistants to enhance the capabilities and flexibility of their generative AI applications.

In [ ]:
#! pip install langchain==0.1.20 langchain_openai sentence-transformers duckduckgo-search gradio
#! pip install pypdf chromadb faiss-cpu

In [ ]:
import os

In [ ]:
# auth replicated here for reference just in case you choose to do something similar
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# enable tracing and set project name
os.environ['LANGCHAIN_TRACING_V2'] = "false"

# uncomment the following two lines before running the cell if you have a Langchain/Langsmith API Key
#os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
#os.environ['LANGCHAIN_TRACING_V2'] = "true"

# set langchain project
os.environ['LANGCHAIN_PROJECT'] = 'May17'

## Schemas
Resources:
* [Structured Output](https://python.langchain.com/v0.1/docs/modules/model_io/chat/structured_output/)

In [ ]:
from typing import Optional, List, Dict, Any
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
# example interview information
interview_text = """My name is Jackson. I'm from Pittsburgh, Pennsylvania, and I graduated college in 2019.
I'm currently working as an administrative assistant at a law firm. I like to go hiking and play video games in my free time."""

In [ ]:
# specify the output schema
class PersonInformation(BaseModel):
    """Structured information about a person extracted from an interview."""

    name: str = Field(default='', description="The person's name.")
    home: str = Field(default='', description="Where the person is from.")
    graduation_year: int = Field(default=None, description="The year the person graduated.")
    job_title: str = Field(default='', description="The person's job title.")
    hobbies: List[str] = Field(default=[], description="The person's hobbies.")


In [ ]:
# use a model with function calling enabled
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_chatllm = model.with_structured_output(PersonInformation)

/Users/bellcs1/miniconda3/envs/textllms/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatOpenAI.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [ ]:
# create human prompt to send
text_prompt = f"Extract the person's name, home, graduation year, job title, and hobbies from the following interview text: \n\n{interview_text}"

# get structured output (hopefully)
structured_info = structured_chatllm.invoke(text_prompt)
structured_info

PersonInformation(name='Jackson', home='Pittsburgh, Pennsylvania', graduation_year=2019, job_title='administrative assistant at a law firm', hobbies=['hiking', 'playing video games'])

In [ ]:
structured_info.dict()

{'name': 'Jackson',
 'home': 'Pittsburgh, Pennsylvania',
 'graduation_year': 2019,
 'job_title': 'administrative assistant at a law firm',
 'hobbies': ['hiking', 'playing video games']}

# Tools
## Function Calling
Resource: [Tool/Function Calling](https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/)

In [ ]:
from langchain_core.tools import tool

In [ ]:
# somewhat optional for this case
class MultiplySchema(BaseModel):
    """Multiply two integer values together. These values must be integers."""

    a: int = Field(..., description="First integer value")
    b: int = Field(..., description="Second integer value")

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

In [ ]:
# define available tools
tools = [multiply]
tool_lookup = {'multiply': multiply}

In [ ]:
# create the model
chatllm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# binds the tools
tool_chatllm = chatllm.bind_tools(tools)

In [ ]:
# use
tool_response = tool_chatllm.invoke("What is 6 times 5?")

# view response
tool_response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_spDMnxZNPW1X9kYiNZeiLw3t', 'function': {'arguments': '{"a":6,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 83, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-55375e45-017e-4958-9339-42e73ef19434-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'call_spDMnxZNPW1X9kYiNZeiLw3t'}])

In [ ]:
tool_response.tool_calls

[{'name': 'multiply',
  'args': {'a': 6, 'b': 5},
  'id': 'call_spDMnxZNPW1X9kYiNZeiLw3t'}]

In [ ]:
# get function that is to be called
fn_to_call = tool_response.tool_calls[0]['name']

# call function
tool_lookup[fn_to_call].invoke(tool_response.tool_calls[0]['args'])

30

### Fitting into the function calling framework
Resource: [Langchain v2 Tool Calling](https://python.langchain.com/v0.2/docs/how_to/tool_calling/)

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

In [ ]:
query = "I have 6 books of 5 pages each. How many pages do I have in total?"
available_tools = {'multiply':multiply}

# get response from llm
llm_response = tool_chatllm.invoke(query)
llm_response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RdnWnbeLGHXtHWoq171PzeFw', 'function': {'arguments': '{"a":6,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 95, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4a87e32f-c4d9-4d26-b88d-41be2f99f49f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'call_RdnWnbeLGHXtHWoq171PzeFw'}])

In [ ]:
# construct message history to complete the rest of the AI Message response
message_history = [HumanMessage(query), llm_response]

# make all tool calls
for tool_call in llm_response.tool_calls:
    # get the name of the called tool from the response
    called_tool_name = tool_call["name"]

    # get the tool object from the available tools
    selected_tool = available_tools[called_tool_name]

    # actually use the tool using the provided arguments
    tool_output = selected_tool.invoke(tool_call["args"])

    # append to the message history
    message_history.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

# view the message history    
message_history

[HumanMessage(content='I have 6 books of 5 pages each. How many pages do I have in total?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RdnWnbeLGHXtHWoq171PzeFw', 'function': {'arguments': '{"a":6,"b":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 95, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4a87e32f-c4d9-4d26-b88d-41be2f99f49f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 6, 'b': 5}, 'id': 'call_RdnWnbeLGHXtHWoq171PzeFw'}]),
 ToolMessage(content='30', tool_call_id='call_RdnWnbeLGHXtHWoq171PzeFw')]

In [ ]:
# finish the call to the AI
final_answer = tool_chatllm.invoke(message_history)
final_answer

AIMessage(content='You have a total of 30 pages.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 120, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e1b1f1cc-6644-43b1-ae9a-752bf02a843e-0')

# Agents
Learn more about [Agents](https://python.langchain.com/docs/modules/agents/quick_start) in their Quickstart

**Chain Tool Invocation**
<figure>
<img src='https://python.langchain.com/v0.1/assets/images/tool_chain-3571e7fbc481d648aff93a2630f812ab.svg' height=300/>
    <figcaption>
        Source: Chain Tool Invocation, from <a href=https://python.langchain.com/v0.1/docs/use_cases/tool_use/>Calling tools with Chains</a>
    </figcaption>
</figure>

**Agent-based Tool Invocation**
<figure>
<img src='https://python.langchain.com/v0.1/assets/images/tool_agent-d25fafc271da3ee950ac1fba59cdf490.svg' height=400/>
    <figcaption>
        Source: Agent-based Tool Invocation, from <a href=https://python.langchain.com/v0.1/docs/use_cases/tool_use//>Calling tools with Agents</a>
    </figcaption>
</figure>


## Using Tools
Resource: [Q&A with RAG -> More -> Using Agents](https://python.langchain.com/v0.1/docs/use_cases/question_answering/conversational_retrieval_agents/)

### Choose Tools

In [ ]:
from langchain.tools import DuckDuckGoSearchResults
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


### Built in tools

In [ ]:
search = DuckDuckGoSearchResults()

# example
search.run("How many children does Barack Obama have?")

"[snippet: Barack and Michelle Obama's daughters, Malia and Sasha, grew up in the White House from 2009 to 2017. To most of the world, Barack and Michelle Obama are the former president and first lady of ..., title: All About Barack and Michelle Obama's 2 Daughters, Malia and Sasha Obama, link: https://people.com/politics/all-about-barack-obama-michelle-obama-daughters/], [snippet: Malia Obama. On June 4, 1998, Obama and Michelle Obama welcomed their first daughter, Malia Ann Obama, into the world. In 2020, Barack Obama talked to InStyle about his daughter's personality ..., title: Who Are Former President Barack Obama's kids? All About Malia ... - TODAY, link: https://www.today.com/parents/barack-obama-kids-rcna134764], [snippet: Watch: Malia Obama Ditches Her Famous Last Name! Malia Obama is voting for a fresh start. As her Hollywood career takes off, Barack and Michelle Obama 's oldest daughter has dropped her last name ..., title: Malia Obama Is Now Going by This Stage Name - E! On

In [ ]:
# create tools
tools = [search]

# create llm with bound tools
search_llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
# create prompt
search_prompt = ChatPromptTemplate.from_messages([
    ('system', """You are a helpful assistant. You ALWAYS use the tools you have available instead of relying on internal information.
     You are brief and succinct in your responses. You use tools first to find information.
     You search the web to find answers about noteworthy figures or recent events."""),
    ('human', "{text}"),
    ('placeholder', "{agent_scratchpad}")
])

# create agent
agent = create_tool_calling_agent(search_llm, tools, search_prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)


In [ ]:
# try it out
response = agent_executor.invoke({'text':"How many children does Barack Obama have?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Barack Obama children'}`


[snippet: Barack and Michelle Obama's daughters, Malia and Sasha, grew up in the White House from 2009 to 2017. To most of the world, Barack and Michelle Obama are the former president and first lady of ..., title: All About Barack and Michelle Obama's 2 Daughters, Malia and Sasha Obama, link: https://people.com/politics/all-about-barack-obama-michelle-obama-daughters/], [snippet: Malia Obama. On June 4, 1998, Obama and Michelle Obama welcomed their first daughter, Malia Ann Obama, into the world. In 2020, Barack Obama talked to InStyle about his daughter's personality ..., title: Who Are Former President Barack Obama's kids? All About Malia ... - TODAY, link: https://www.today.com/parents/barack-obama-kids-rcna134764], [snippet: The former FLOTUS shares her two daughters with husband Barack Obama. Hanna Fillingham . US Managing Editor. 3 days ago 10 May 2024, 18:46 

### Custom tools

In [ ]:
# create tools
tools = [search, multiply]

# create agent
agent = create_tool_calling_agent(search_llm, tools, search_prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

# try it out
response = agent_executor.invoke({'text':"""How many grand children would Barack Obama have if each of his children had 2 children?
                                  I heard he recently had a baby!"""})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 2}`


4
Invoking: `duckduckgo_results_json` with `{'query': 'Barack Obama recent baby'}`


[snippet: Barack and Michelle Obama are celebrating their eldest daughter. The former first couple took to Instagram on Tuesday to honor Malia Obama on her 25th birthday. The pair also shares Sasha, 22., title: Barack Obama Honors 'Talented, Hilarious, and Beautiful' Daughter Malia ..., link: https://www.etonline.com/barack-obama-honors-talented-hilarious-and-beautiful-daughter-malia-on-her-25th-birthday-207428], [snippet: Barack and Michelle Obama's daughters, Malia and Sasha, grew up in the White House from 2009 to 2017. To most of the world, Barack and Michelle Obama are the former president and first lady of ..., title: All About Barack and Michelle Obama's 2 Daughters, Malia and Sasha Obama, link: https://people.com/politics/all-about-barack-obama-michelle-obama-daughters/], [snippet: Former first lady and wife

## Retrievers as tools

In [ ]:
from bs4 import SoupStrainer
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Creating custom retriever tools
from langchain.tools.retriever import create_retriever_tool

In [ ]:
# use website data
sw_website = 'https://simple.wikipedia.org/wiki/Star_Wars_Episode_IV:_A_New_Hope'
webloader = WebBaseLoader(sw_website,
                       bs_kwargs = {'parse_only':SoupStrainer('div', id='bodyContent')})
web_chunks = webloader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True))
print('Number of chunks generated: ', len(web_chunks))

# create embeddings
embeddings_fn = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") #, model_kwargs={"device":'mps'})
hf_db = FAISS.from_documents(web_chunks, embeddings_fn)
hf_retriever = hf_db.as_retriever(search_kwargs={"k":5})

Number of chunks generated:  17


In [ ]:
tool_name = "Information Retriever for the Star Wars Movie: A New Hope"
description = """You MUST use this tool whenever any information is requested about the Star Wars movie: A New Hope.
    Make sure to rephrase the query so that it is not a question, but a statement to be compared with
    text about the movie."""

# based off of the Guide we used
retriever_tool = create_retriever_tool(
    hf_retriever,
    tool_name,
    description
)

In [ ]:
# define tools
tools = [search, retriever_tool]

In [ ]:
# create prompt
react_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(template='Answer the following questions as best you can or reply with the most meaningful ' +
                                             'response possible. You have access to the following tools:\n\n{tools}\n\n although you can also reply conversationally when appropriate. '
                                             'Use the following format to reason out your response:\n\n' +
                                             'Question: the input question you must answer or statement to which you should reply\n' +
                                             'Thought: you should always think about what to do\n' +
                                             'Action: the action to take, should be one of [{tool_names}]\n'+
                                             'Action Input: the input to the action, making sure that the inputs are in the valid format for the action\n'+
                                             'Observation: the result of the action\n... '+
                                             '(this Thought/Action/Action Input/Observation can repeat up to 3 times)\n'+
                                             'Thought: I now know the final answer\n'+
                                             'Final Answer: the final answer to the original input question or appropriate response is\n\n'+
                                             'Begin!\n\nQuestion or message: {input}\nThought:{agent_scratchpad}'),
  ]
)

In [ ]:
# Create our own llm and agent
agent = create_react_agent(model, tools, react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, run_intermediate_steps=True)

In [ ]:
# Execute the agent
agent_executor.invoke(
    {"input":"What happened to Princess Leia in a New Hope?"}
)



> Entering new AgentExecutor chain...
I need to retrieve information about Princess Leia in the movie A New Hope.
Action: Information Retriever for the Star Wars Movie: A New Hope
Action Input: Princess Leia in A New HopePlot[change | change source]
The film takes place years after Revenge of the Sith, which was made in 2005 as a prequel. The Rebellion has stolen the secret plans for the Galactic Empire's superweapon known as the Death Star. Darth Vader and his stormtroopers capture Princess Leia Organa, a leader of the Rebellion but she secretly gives the Death Star plans to two droids (robots), C-3PO and R2-D2.

Leia. They eventually find her in a cell. As they try to escape, Darth Vader kills Obi-Wan in a lightsaber battle, but the rest of the heroes escape on the Millennium Falcon.

Star Wars: Episode IV: A New Hope (initially named Star Wars) is a science fiction movie. It is the first film made in the Star Wars saga, but is the fourth film in the story's timeline. The movie was

{'input': 'What happened to Princess Leia in a New Hope?',
 'output': 'Princess Leia was captured by Darth Vader in A New Hope and played a crucial role in getting the Death Star plans to the Rebellion.'}

In [ ]:
query = """What did Obi-Wan do for Luke Skywalker in a New Hope?
How did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies."""

# Execute the agent
agent_executor.invoke(
    {"input":query}
)



> Entering new AgentExecutor chain...
I need to find out what Obi-Wan did for Luke Skywalker in A New Hope and then see how it influenced the outcome of The Empire Strikes Back.
Action: Information Retriever for the Star Wars Movie: A New Hope
Action Input: Obi-Wan's actions for Luke Skywalker in A New HopeObi-Wan explains that the Jedi used a power known as the Force to keep peace in the galaxy until the Empire hunted them down. R2-D2 shows Obi-Wan the distress call from Princess Leia. Luke finds out that stormtroopers have destroyed his home, including his aunt and uncle, in search for R2-D2. Luke decides to accompany Obi-Wan and the droids to Alderaan, Leia's homeworld.

Star Wars: Episode IV: A New Hope (initially named Star Wars) is a science fiction movie. It is the first film made in the Star Wars saga, but is the fourth film in the story's timeline. The movie was released in 1977 and also incorporates adventure, action and drama.

Wikiquote has a collection of quotations rela

{'input': 'What did Obi-Wan do for Luke Skywalker in a New Hope?\nHow did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies.',
 'output': "Obi-Wan's actions in A New Hope, including his sacrifice, influenced the outcome of The Empire Strikes Back by shaping Luke's path and the eventual defeat of the Galactic Empire."}

## Adding the conversational component

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# start with empty chat history. We can pass this as a parameter in the invoke
message_history = ChatMessageHistory()

# Create the agent with chat history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    handle_parsing_errors = "Check your output and make sure it conforms, use the Action/Action Input syntax"
)

In [ ]:
# Start chatting away
query_1 = """What did Obi-Wan do for Luke Skywalker in a New Hope?
How did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies."""

session_id_1 = "session_1"

# Execute the agent
agent_with_chat_history.invoke({"input":query_1}, config={"configurable": {"session_id": session_id_1}})

Parent run 2d79c8d7-9200-443c-aa6e-41f1214298ac not found for run b3fdbbdf-9cb9-4b8b-a958-309c5425074e. Treating as a root run.




> Entering new AgentExecutor chain...
I need to gather information about Obi-Wan's actions in A New Hope and how they influenced the outcome of The Empire Strikes Back.
Action: Information Retriever for the Star Wars Movie: A New Hope
Action Input: Obi-Wan's actions in A New HopeObi-Wan explains that the Jedi used a power known as the Force to keep peace in the galaxy until the Empire hunted them down. R2-D2 shows Obi-Wan the distress call from Princess Leia. Luke finds out that stormtroopers have destroyed his home, including his aunt and uncle, in search for R2-D2. Luke decides to accompany Obi-Wan and the droids to Alderaan, Leia's homeworld.

Star Wars: Episode IV: A New Hope (initially named Star Wars) is a science fiction movie. It is the first film made in the Star Wars saga, but is the fourth film in the story's timeline. The movie was released in 1977 and also incorporates adventure, action and drama.

Wikiquote has a collection of quotations related to: Star Wars Episode IV

{'input': 'What did Obi-Wan do for Luke Skywalker in a New Hope?\nHow did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies.',
 'chat_history': [],
 'output': "Obi-Wan's actions in A New Hope set the stage for key plot points in The Empire Strikes Back, such as the revelation of Darth Vader as Luke's father and the impact of Obi-Wan's character development."}

In [ ]:
# Start chatting away
query_2 = """Elaborate on your previous answer, and provide details that reference the object that Obi-Wan gave to Luke Skywalker in A New Hope."""

# Execute the agent
agent_with_chat_history.invoke({"input":query_2}, config={"configurable": {"session_id": session_id_1}})

Parent run cf23909f-311a-4fc3-9b18-73b65dc492e4 not found for run a5b95173-cf20-4c54-83b7-fcd85530ad9d. Treating as a root run.




> Entering new AgentExecutor chain...
I should use the Information Retriever for the Star Wars Movie: A New Hope to find out more about the object that Obi-Wan gave to Luke Skywalker in the movie.

Action: Information Retriever for the Star Wars Movie: A New Hope
Action Input: Object given to Luke Skywalker by Obi-Wan in A New HopeObi-Wan explains that the Jedi used a power known as the Force to keep peace in the galaxy until the Empire hunted them down. R2-D2 shows Obi-Wan the distress call from Princess Leia. Luke finds out that stormtroopers have destroyed his home, including his aunt and uncle, in search for R2-D2. Luke decides to accompany Obi-Wan and the droids to Alderaan, Leia's homeworld.

They arrive at the Rebel base and R2-D2 gives the secret plans to the Rebels and they discover a weakness in the Death Star. They organise an attack on the Death Star in their small ships called starfighters. With the help of the Force, Luke is able to destroy the Death Star. The Rebellion

{'input': 'Elaborate on your previous answer, and provide details that reference the object that Obi-Wan gave to Luke Skywalker in A New Hope.',
 'chat_history': [HumanMessage(content='What did Obi-Wan do for Luke Skywalker in a New Hope?\nHow did this influence affect the outcome of the Empire Strikes Back? You can use the web to find out more about other movies.'),
  AIMessage(content="Obi-Wan's actions in A New Hope set the stage for key plot points in The Empire Strikes Back, such as the revelation of Darth Vader as Luke's father and the impact of Obi-Wan's character development.")],
 'output': 'In A New Hope, Obi-Wan gave Luke Skywalker a lightsaber, which was a key weapon and symbol of the Jedi Order.'}

# Bringing it all together
With a user history, a tool, and a retriever, you can now create a conversational agent through a quick UI that can answer questions and provide information.

In [ ]:
import gradio as gr

In [ ]:
message_history.clear()
session_id_chat = 'session_id_chat'

In [ ]:
# create response
def ai_response(user_message, chat_info):
    agent_message = {'input':user_message}
    response = agent_with_chat_history.invoke(agent_message, config={"configurable": {"session_id": session_id_chat}})
    conversation = message_history.messages
    formatted_messages = [(conversation[ind].content, conversation[ind+1].content)
                           for ind in range(0, len(conversation), 2)]
    return '', formatted_messages

# create gradio blocks UI
with gr.Blocks() as demo :
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(ai_response, [msg, chatbot], [msg, chatbot])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# OpenAI Assistants - Agents! (?)
Resource: [Assistants API](https://platform.openai.com/docs/assistants/overview)

Resource: [Platform Documentation - time for the playground!](https://platform.openai.com/playground/assistants)

## Conversation

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# create assistant with code interpreter
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter", "type": "file_search"}],
    model="gpt-4o",
)

IMPORTANT: You are using gradio version 4.19.2, however version 4.29.0 is available, please upgrade.
--------


EASY message history!

In [ ]:
# create a thread of message history
thread = client.beta.threads.create()
thread

Thread(id='thread_w9jVDGpzd4hBLGX0rsrQmfdF', created_at=1715928365, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [ ]:
# add a message to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
message

Message(id='msg_3WaUjfPYMiSmZDKoO733U5Ud', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1715928365, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_w9jVDGpzd4hBLGX0rsrQmfdF')

In [ ]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="The user prefers to have concepts explained to them like they're 5. Then, describe it at an appropriate college level."
)

In [ ]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print('Finished running.')
else:
  print(run.status)

messages.model_dump()

Finished running.


{'data': [{'id': 'msg_wdsp54KkeXkuOSwGGk5hNhaE',
   'assistant_id': 'asst_Vyo3OdCTDib9zeAFqnmoOckJ',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': "Sure, I can help you with this!\n\n### Simple Explanation (like you're 5)\nImagine you have 3 bags of cookies (each bag is represented by `x`) and 11 loose cookies. If you put them all together, you end up with 14 cookies. To find out how many cookies are in each bag, we need to solve the equation `3x + 11 = 14`.\n\n#### Step-by-Step:\n1. You have `3x` bags and you want to find out how many cookies are in each bag if you already know you have 11 more cookies making the total 14.\n2. First, take away those 11 loose cookies from the total of 14 cookies.\n   - \\( 14 - 11 = 3 \\)\n3. Now, you are left with 3 cookies, which are divided into the 3 bags (`3x`).\n4. To know how many cookies are in each bag, you divide those 3 cookies by 3.\n   - \\( 3 \\div 3 = 1 \\)\nSo, each bag has 1 c

In [ ]:
# get message information
print(len(messages.data))

2


In [ ]:
# view the ai response
print(messages.data[0].content[0].text.value)

Sure, I can help you with this!

### Simple Explanation (like you're 5)
Imagine you have 3 bags of cookies (each bag is represented by `x`) and 11 loose cookies. If you put them all together, you end up with 14 cookies. To find out how many cookies are in each bag, we need to solve the equation `3x + 11 = 14`.

#### Step-by-Step:
1. You have `3x` bags and you want to find out how many cookies are in each bag if you already know you have 11 more cookies making the total 14.
2. First, take away those 11 loose cookies from the total of 14 cookies.
   - \( 14 - 11 = 3 \)
3. Now, you are left with 3 cookies, which are divided into the 3 bags (`3x`).
4. To know how many cookies are in each bag, you divide those 3 cookies by 3.
   - \( 3 \div 3 = 1 \)
So, each bag has 1 cookie. Therefore, \( x = 1 \).

### Detailed Explanation (College Level)
Given the linear equation `3x + 11 = 14`, we need to isolate the variable `x` to solve for its value.

#### Step-by-Step:
1. **Subtract 11 from both sid

In [ ]:
# view all messages
for ind, message in enumerate(messages.data):
    print(f'***** Message {ind}:\n{message.content[0].text.value}\n *****')

***** Message 0:
Sure, I can help you with this!

### Simple Explanation (like you're 5)
Imagine you have 3 bags of cookies (each bag is represented by `x`) and 11 loose cookies. If you put them all together, you end up with 14 cookies. To find out how many cookies are in each bag, we need to solve the equation `3x + 11 = 14`.

#### Step-by-Step:
1. You have `3x` bags and you want to find out how many cookies are in each bag if you already know you have 11 more cookies making the total 14.
2. First, take away those 11 loose cookies from the total of 14 cookies.
   - \( 14 - 11 = 3 \)
3. Now, you are left with 3 cookies, which are divided into the 3 bags (`3x`).
4. To know how many cookies are in each bag, you divide those 3 cookies by 3.
   - \( 3 \div 3 = 1 \)
So, each bag has 1 cookie. Therefore, \( x = 1 \).

### Detailed Explanation (College Level)
Given the linear equation `3x + 11 = 14`, we need to isolate the variable `x` to solve for its value.

#### Step-by-Step:
1. **Subtract

In [ ]:
# view entire run information
run.model_dump()

{'id': 'run_FjpAQTJvs0gN5HyvWvCEeuec',
 'assistant_id': 'asst_Vyo3OdCTDib9zeAFqnmoOckJ',
 'cancelled_at': None,
 'completed_at': 1715928372,
 'created_at': 1715928366,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': "The user prefers to have concepts explained to them like they're 5. Then, describe it at an appropriate college level.",
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1715928366,
 'status': 'completed',
 'thread_id': 'thread_w9jVDGpzd4hBLGX0rsrQmfdF',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search'}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 415,
  'prompt_tokens': 625,
  'total_tokens': 1040},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

## Tools: Code Interpreter
Resource: [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter)

In [ ]:
! curl -o palmer_penguins.csv https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100 15241  100 15241    0     0   123k      0 --:--:-- --:--:-- --:--:--  124k


In [ ]:
# Upload the penguins file to be used directly with a message thread with code interpreter
palmer_penguins_data = client.files.create(
    file=open("palmer_penguins.csv", "rb"),
    purpose='assistants'
)

In [ ]:
# modify thread to include the penguins file
ci_thread = client.beta.threads.create(
    tool_resources={"code_interpreter": {"file_ids": [palmer_penguins_data.id]}}
)

In [ ]:
# create new message to call palmer penguins
message = client.beta.threads.messages.create(
    thread_id=ci_thread.id,
    role="user",
    content="I need to analyze the Palmer Penguins data I uploaded. Create a histogram reflecting the counts of the different species of penguins.",
    attachments=[
        {
            "file_id":palmer_penguins_data.id,
            "tools":[{"type":"code_interpreter"}]
        }
    ]
)

In [ ]:
# create run and force use of code interpreter
run = client.beta.threads.runs.create_and_poll(
  thread_id=ci_thread.id,
  assistant_id=assistant.id,
  tools=[{"type":"code_interpreter"}],
  tool_choice = {"type":"code_interpreter"}
)

In [ ]:
# check completion and view output
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=ci_thread.id
  )
  print('Finished running.')
else:
  print(run.status)

messages.model_dump()
#run.model_dump()

Finished running.


{'data': [{'id': 'msg_VqZdA7s9skm0NORphTKlTpZG',
   'assistant_id': 'asst_Vyo3OdCTDib9zeAFqnmoOckJ',
   'attachments': [],
   'completed_at': None,
   'content': [{'image_file': {'file_id': 'file-upfdqadwWqJvYXRztNU3gW8z',
      'detail': None},
     'type': 'image_file'},
    {'text': {'annotations': [],
      'value': 'The histogram above depicts the counts of different penguin species in the Palmer Penguins dataset. If you need further analysis or assistance with specific questions, feel free to ask!'},
     'type': 'text'}],
   'created_at': 1715928434,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_z9gdHm5geOrLBUuqbbwjgWBZ',
   'status': None,
   'thread_id': 'thread_U076ZeuqmWHxPnGoZOglhpWR'},
  {'id': 'msg_JLaEniKfbW5PkVpRTSDNifMq',
   'assistant_id': None,
   'attachments': [{'file_id': 'file-XjCZqsmdOee1S3ckA4npOZiC',
     'tools': [{'type': 'code_interpreter'}]}],
   'complet

In [ ]:
len(messages.data)

2

In [ ]:
# retrieve a single image file
#generated_image = messages.data[0].content[0].image_file.file_id
#print(generated_image)

# download using file api
#image_data = client.files.content(generated_image)
#image_data_bytes = image_data.read()

file-upfdqadwWqJvYXRztNU3gW8z


## Retrieval and Vector Stores
Resource: [File Search](https://platform.openai.com/docs/assistants/tools/file-search/file-search-beta)

In [ ]:
! curl -o declaration_independence.pdf https://www.uscis.gov/sites/default/files/document/guides/M-654.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100  724k  100  724k    0     0  3605k      0 --:--:-- --:--:-- --:--:-- 3624k


In [ ]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="declaration of independence")
 
# Ready the files for upload to OpenAI
file_paths = ["declaration_independence.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [ ]:
# create assistant with code interpreter
history_assistant = client.beta.assistants.create(
    name="Prominent H. Figure",
    instructions="You are a prominent historical figure from the late 1770s. You're an opinionated dynamic oratorical speaker, but you make your point succinctly!",
    tools=[{"type": "file_search"}],
    model="gpt-3.5-turbo",
)

In [ ]:
history_assistant = client.beta.assistants.update(
  assistant_id=history_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [ ]:
# create thread for history
history_thread = client.beta.threads.create()
history_thread

Thread(id='thread_vYbgwwa3Pq6SZwY6z4StBNqU', created_at=1715928441, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [ ]:
# create and send message
message = client.beta.threads.messages.create(
  thread_id=history_thread.id,
  role="user",
  content="What reasons do the authors cite for declaring independence from Great Britain in the Declaration of Independence?"
)
message

Message(id='msg_dPuZNXgFV8W0JN21PlCykomh', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What reasons do the authors cite for declaring independence from Great Britain in the Declaration of Independence?'), type='text')], created_at=1715928441, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_vYbgwwa3Pq6SZwY6z4StBNqU')

In [ ]:
# create run and force use of code interpreter
run = client.beta.threads.runs.create_and_poll(
  thread_id=history_thread.id,
  assistant_id=history_assistant.id,
)

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=history_thread.id
  )
  print('Finished running.')
else:
  print(run.status)

Finished running.


In [ ]:
#view result
messages.model_dump()

{'data': [{'id': 'msg_yhI0PRWzcRREpVAKY4uQaP0v',
   'assistant_id': 'asst_vi4H4n9ajWiYZTWhMBP16dGY',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [{'end_index': 1267,
        'file_citation': {'file_id': 'file-zCl01AgCTv3P6hnNfWk6i7et',
         'quote': None},
        'start_index': 1255,
        'text': '【4:0†source】',
        'type': 'file_citation'},
       {'end_index': 1279,
        'file_citation': {'file_id': 'file-zCl01AgCTv3P6hnNfWk6i7et',
         'quote': None},
        'start_index': 1267,
        'text': '【4:3†source】',
        'type': 'file_citation'}],
      'value': "The authors of the Declaration of Independence cited several reasons for declaring independence from Great Britain. These reasons included:\n\n1. Imposing standing armies on the colonies without consent of the legislatures\n2. Making the military independent of and superior to civil power\n3. Subjecting the colonies to foreign jurisdiction and laws not recognized b

# Congratulations!! 
You have now learned how to use tools, agents, and assistants to enhance the capabilities of your generative AI applications. You have experience using the OpenAI Completions and Assistants API, are able to execute RAG with conversation history, and are able to port this knowledge to also using agents and tools.

Next week, we'll start model training - congratulations again!